In [4]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
import pickle

In [ ]:

file_list = []
class_list = []

DATADIR = r'D:\plants-images'

# All the categories you want your neural network to detect
CATEGORIES = ['flower', 'foliage', 'succulent']

# The size of the images that your neural network will use
IMG_SIZE = 50

# Checking or all images in the data folder
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    for plant in os.listdir(path):
        path_fig = os.path.join(path, plant)
        for img in os.listdir(path_fig):
            img_array = cv2.imread(os.path.join(path_fig, img), cv2.IMREAD_GRAYSCALE)

training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for plant in os.listdir(path):
            path_fig = os.path.join(path, plant)
            for img in os.listdir(path_fig):
                try :
                    img_array = cv2.imread(os.path.join(path_fig, img), cv2.IMREAD_GRAYSCALE)
                    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                    training_data.append([new_array, class_num])
                except Exception as e:
                    pass

create_training_data()

random.shuffle(training_data)

X = [] #features
y = [] #labels

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

# Creating the files containing all the information about your model
pickle_out = open('X.pickle', 'wb')
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open('y.pickle', 'wb')
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in = open('X.pickle', 'rb')
X = pickle.load(pickle_in)

In [9]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
from keras.models import model_from_json
from keras.models import load_model
import matplotlib.pyplot as plt

In [11]:
with tf.device('/device:XLA_GPU:0'):

	# Opening the files about data
	X = pickle.load(open("X.pickle", "rb"))
	y = pickle.load(open("y.pickle", "rb"))

	# normalizing data (a pixel goes from 0 to 255)
	X = X/255.0

	from sklearn.model_selection import train_test_split
	import numpy as np
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
	X_train = np.asarray(X_train)
	X_test = np.asarray(X_test)
	y_train = np.asarray(y_train)
	y_test = np.asarray(y_test)


	# Building the model
	model = Sequential()
	# 3 convolutional layers
	model.add(Conv2D(32, (3, 3), input_shape = X.shape[1:]))
	model.add(Activation("relu"))
	model.add(MaxPooling2D(pool_size=(2,2)))

	model.add(Conv2D(64, (3, 3)))
	model.add(Activation("relu"))
	model.add(MaxPooling2D(pool_size=(2,2)))

	model.add(Conv2D(64, (3, 3)))
	model.add(Activation("relu"))
	model.add(MaxPooling2D(pool_size=(2,2)))
	model.add(Dropout(0.25))

	# 2 hidden layers
	model.add(Flatten())
	model.add(Dense(128))
	model.add(Activation("relu"))

	model.add(Dense(128))
	model.add(Activation("relu"))

	# The output layer with 3 neurons, for 3 classes
	model.add(Dense(3))
	model.add(Activation("softmax"))

	# Compiling the model using some basic parameters
	model.compile(loss="sparse_categorical_crossentropy",
					optimizer="adam",
					metrics=["accuracy"])

	# Training the model, with 40 iterations
	# validation_split corresponds to the percentage of images used for the validation phase compared to all the images
	history = model.fit(X_train, y_train, batch_size=32, epochs=40, validation_data=(X_test, y_test))

	# Saving the model
	model_json = model.to_json()
	with open("model.json", "w") as json_file :
		json_file.write(model_json)

	model.save_weights("model.h5")
	print("Saved model to disk")

	model.save('CNN.model')

	# Printing a graph showing the accuracy changes during the training phase
	print(history.history.keys())
	plt.figure(1)
	plt.plot(history.history['acc'])
	plt.plot(history.history['val_acc'])
	plt.title('model accuracy')
	plt.ylabel('accuracy')
	plt.xlabel('epoch')
	plt.legend(['train', 'validation'], loc='upper left')

Epoch 1/40


InvalidArgumentError: Function invoked by the following node is not compilable: {{node __inference_train_function_2003}} = __inference_train_function_2003[_XlaMustCompile=true, config_proto="\n\007\n\003CPU\020\001\n\007\n\003GPU\020\0002\002J\0008\001\202\001\000", executor_type=""](dummy_input, dummy_input, dummy_input, dummy_input, dummy_input, dummy_input, dummy_input, dummy_input, dummy_input, dummy_input, ...).
Uncompilable nodes:
IteratorGetNext: unsupported op: No registered 'IteratorGetNext' OpKernel for XLA_CPU_JIT devices compatible with node {{node IteratorGetNext}}
	Stacktrace:
		Node: __inference_train_function_2003, function: 
		Node: IteratorGetNext, function: __inference_train_function_2003
 [Op:__inference_train_function_2003]

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  1


In [7]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6882097163751405194
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9597274238103880697
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12260387271391005505
physical_device_desc: "device: XLA_GPU device"
]
